In [ ]:
#https://www.kdnuggets.com/2020/02/audio-data-analysis-deep-learning-python-part-1.html

In [ ]:
import librosa
import pandas as pd
import numpy as np
from google.colab import files
import os
import shutil
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')
import os
import itertools
import random

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
test_image = '/content/drive/Shared drives/SpotifyMemoria/dog.jpeg'

## Pruebas modelo preentrenado

In [ ]:
# example of loading the resnet50 model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Flatten, Dense
# load model
model_resnet = ResNet50()
flat1 = Flatten()(model_resnet.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(10, activation='softmax')(class1)
# summarize the model

102981632/102967424 [==============================] - 0s 0us/step


In [ ]:
model = Model(inputs=model_resnet.input, outputs=output)

## Pre-Trained Model as Classifier


In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
# load an image from file
image = load_img(test_image, target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# load the model
model = VGG16()
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

49152/35363 [=========================================] - 0s 0us/step
golden_retriever (25.12%)


## Pre-Trained Model as Feature Extractor Preprocessor

In [ ]:
# example of using the vgg16 model as a feature extraction model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from pickle import dump
# load an image from file
image = load_img(test_image, target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
# get extracted features
features = model.predict(image)
print(features.shape)
# save to file
dump(features, open('dog.pkl', 'wb'))

(1, 4096)


In [ ]:
## Pre-Trained Model as Feature Extractor in Model

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(300, 300, 3))
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(10, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
#model.summary()

58900480/58889256 [==============================] - 0s 0us/step


### GLOBALS

In [ ]:
images_full = '/content/drive/Shared drives/SpotifyMemoria/Cover_art_dataset/'

audio_path = '/content/drive/Shared drives/SpotifyMemoria/MM_dataset/train/audio/'

image_path =  '/content/drive/Shared drives/SpotifyMemoria/MM_dataset/train/image/'

spect_path = '/content/drive/Shared drives/SpotifyMemoria/MM_dataset/train/audio_spectrogram/'

In [ ]:
genres_y = [ 'blues', 'bluegrass', 'death-metal','ambient', 'alt-rock', 'afrobeat', 'acoustic', 'club', 'chill', 'deep-house', 'alternative', 'black-metal', 'breakbeat', 'british', 'cantopop', 'disco', 'edm']

In [ ]:
'''
#Sacar los que estaban repetidos en test con train 
path_aud_test = '/content/drive/Shared drives/SpotifyMemoria/MM_dataset/test/audio/'
path_repetidos = '/content/drive/Shared drives/SpotifyMemoria/MM_dataset/Repetidos/'
test_aud = os.listdir('/content/drive/Shared drives/SpotifyMemoria/MM_dataset/test/audio/')

for genre in test_aud:
  print(aud)
  test_folder_genre = os.listdir(path_aud_test+genre)
  for file in test_folder_genre:
    aud_file = path_aud_test+genre+'/'+file
    aud_train = aud_file.replace("test", "train")
    if os.path.exists(aud_train):
      shutil.move(aud_file, path_repetidos)
      print('test', aud_file)
      print('train', aud_train)
'''

In [ ]:
image_path =  '/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/test/image/'

In [ ]:
import cv2


for genre in genres_y:
  cont = 0

  image_folder = os.listdir(image_path+genre)
  for file in image_folder:
    image = cv2.imread(image_path+genre+'/'+file)
    if image.any() != None:
      if(len(image.shape)<2):
            print ('grayscale')
            print(image_path+genre+'/'+file)
      elif len(image.shape)==3:
            #print(image.shape[2])
            continue

    else:
      print("cannot find image") 
  

### Functions

In [ ]:
import matplotlib.pyplot as plt
import librosa
import librosa.display

def spectrogram(file_name):
    x, sr = librosa.load(file_name)
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    return tf.convert_to_tensor(Xdb)

In [ ]:
#Extract spectrogram from every audio
audio_path_2 = '/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio/'
spect_path_2 =  '/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/'

cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
#genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
genres = os.listdir(audio_path_2)
for g in genres:
    a_path= g 
    songs = os.listdir(audio_path_2+a_path)
    if not os.path.exists(spect_path_2+g):
  
        # Create a new directory because it does not exist 
        os.makedirs(spect_path_2+g)
    for j in songs:

        spotify_id = j.split('.')[0]
        spect_file = spotify_id + '.jpg'
        if os.path.exists(spect_path_2+g+'/'+spect_file):
          continue

        y, sr = librosa.load(audio_path_2+g+'/'+j, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        print(spect_path_2+g+'/'+spect_file)
        plt.savefig(spect_path_2+g+'/'+spect_file)
        plt.clf()


/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/electro/0NNgY2gc1TUwBQoVp98Fxw.jpg
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/electro/0KmgJyW4GDgonqsoyx0CZ3.jpg
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/electro/1OzxzCYQkJNvj6IHcA2Atc.jpg
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/electro/1rHmaLkF7ZvUmYOIIbbJpn.jpg
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/electro/1lBy6fjbqe5RkGy59zdZ4R.jpg
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/electro/1eqDrhKuCrdcFID3cAY9vw.jpg
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/electro/1AOGhyzZNbRGQ1NCC015LB.jpg
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spectrogram/electro/1HHnQYqcnlnE9q7LGFFGFj.jpg
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio_spe

<Figure size 576x576 with 0 Axes>

In [ ]:
def req_cover_img(img_url,file_ruta):
    response = requests.get(img_url)
    print(response.status_code)
    #file_ruta = base_url + row['genre'] + '/' + row['id_spotify']+".jpg"
    print(file_ruta)
    file = open(file_ruta,"wb+")
    file.write(response.content)
    file.close()

In [ ]:
drive_path = '/content/drive/Shared drives/SpotifyMemoria/'
image_path_2 = '/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/image/'
audio_path_2 = '/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/audio/'

In [ ]:
 #agregar cover art faltante 
genres = os.listdir(audio_path_2)

for i in genres:
    a_path= i 
    songs = os.listdir(audio_path_2+a_path)
    for j in songs:
      spotify_id = j.split('.')[0]
      image_file = spotify_id + '.jpg'

      if(not os.path.exists(image_path_2 + i + '/' + image_file)):
        print("Este tema no tiene cover art")
        spotify = pd.read_csv(drive_path+'spotify_dataset.csv')

        print((spotify.loc[spotify['id_spotify'] == spotify_id]).columns)
        try:
          image_url = spotify.loc[spotify['id_spotify'] == spotify_id,'image_url'].values[0]
        except: 
          continue
        print(image_url)
        req_cover_img(image_url, image_path_2+i+'/'+image_file)
        continue
      else:
        print(spotify_id)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
https://i.scdn.co/image/ab67616d0000b273b79e668c79aed7f6ed35d20e
200
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/image/electronic/4BRa0vbg45PS43pkMsnXyb.jpg
Este tema no tiene cover art
Index(['id_spotify', 'name', 'artist', 'image_url', 'album', 'genre'], dtype='object')
https://i.scdn.co/image/ab67616d0000b27354405f4e966b1efbebb88883
200
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/image/electronic/4Pz7iN649aBn2TwcxBRF8Z.jpg
Este tema no tiene cover art
Index(['id_spotify', 'name', 'artist', 'image_url', 'album', 'genre'], dtype='object')
https://i.scdn.co/image/ab67616d0000b273a26e694a8205807fc1de9bce
200
/content/drive/Shared drives/SpotifyMemoria/MM_full/splits/val/image/electronic/3rkge8kur9i26zpByFKvBu.jpg
Este tema no tiene cover art
Index(['id_spotify', 'name', 'artist', 'image_url', 'album', 'genre'], dtype='object')
https://i.scdn.co/image/ab67616d0000b273c44aa6bac605e42c0

In [ ]:
#Energy function
from keras import backend as K
def euclidean_distance(vects):
   x, y = vects
   return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

#Esto es distancia, asi que entre menor mas parecidos son
#en este caso, deeria etiquetar como 0 los iguales y como 1 los diferentes 

In [ ]:
#from keras import backend as K
#def contrastive_loss(y_true, y_pred):
#   margin = 1
#   return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


import pickle as pkl
from tensorflow import keras
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import tensorflow.keras.backend as K

def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

def contrastive_loss(y, preds, margin=1):
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and the predicted labels
    squaredPreds = K.square(preds) # Al cuadrado...
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    
    loss = K.mean((1 - y) * K.square(preds) + y * K.square(K.maximum(margin - preds, 0)))
    return loss

In [ ]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [ ]:
def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

In [ ]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten,Concatenate,Reshape
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop

import tensorflow as tf

### Models

In [ ]:
def image_feat_network(input_shape):
   seq = Sequential()
   seq.add(Convolution2D(32, kernel_size=(3,3), input_shape=input_shape))
   seq.add(MaxPooling2D(pool_size=(2, 2)))
   seq.add(Dropout(.25))
   seq.add(Convolution2D(32, kernel_size=(4,2), input_shape=input_shape))
   seq.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
   seq.add(Dropout(.25))
   seq.add(Convolution2D(32, kernel_size=(3,3), input_shape=input_shape))
   seq.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
   seq.add(Dropout(.25))
   seq.add(Convolution2D(32, kernel_size=(3,2), input_shape=input_shape))
   seq.add(Activation('relu'))
   seq.add(MaxPooling2D(pool_size=(1, 1)))
   seq.add(Dropout(.25))
   return seq

In [ ]:
def audio_feat_network(input_shape):
    seq = Sequential()
    seq.add(Convolution2D(32, kernel_size=(3,3), input_shape=input_shape))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(4, 2)))
    seq.add(Dropout(.25))
    seq.add(Convolution2D(32, kernel_size=(3,3)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(4, 2)))
    seq.add(Convolution2D(32, kernel_size=(4,1)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(1, 1)))
    seq.add(Convolution2D(32, kernel_size=(4,1)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(1, 1)))
    seq.add(Convolution2D(32, kernel_size=(3,1)))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 1)))
    return seq

In [ ]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten,Concatenate,Reshape
from keras.models import Sequential, Model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization

import tensorflow as tf

In [ ]:
def multi_modal_network(input_shape):
   kernel_size = 3
   seq = Sequential()
   seq.add(Convolution2D(6, kernel_size=(3,3), input_shape=(27, 350, 32)))
   seq.add(Activation('relu'))
   seq.add(MaxPooling2D(pool_size=(2, 2)))
   seq.add(Dropout(.25))
   seq.add(Flatten())
   seq.add(Dense(32, activation='relu'))
   seq.add(Dropout(0.1))
   seq.add(Dense(50, activation='relu'))
   return seq

In [ ]:
def siamese_network(input_dim_img,input_dim_aud):
    
    img_a = Input(shape=input_dim_img)
    img_b = Input(shape=input_dim_img)
    aud_a = Input(shape=input_dim_aud)
    aud_b = Input(shape=input_dim_aud)

    img_network = image_feat_network(input_dim_img)
    feat_img_a = img_network(img_a)
    feat_img_b = img_network(img_b)
    
    aud_network = audio_feat_network(input_dim_aud)
    feat_aud_a = aud_network(aud_a)
    feat_aud_b = aud_network(aud_b)
    
    concat_a = Concatenate(axis=2)([feat_img_a, feat_aud_a])
    concat_b = Concatenate(axis=2)([feat_img_b, feat_aud_b])
    print(concat_a.shape)
    
    input_dim = (27,3,32)
    base_network = multi_modal_network(input_dim)
    feat_vecs_a = base_network(concat_a)
    feat_vecs_b = base_network(concat_b)

    #distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a ,feat_vecs_b])
    distance = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([feat_vecs_a ,feat_vecs_b])
    
    rms = RMSprop()
    model = Model(inputs=[img_a, aud_a, img_b,aud_b], outputs=distance)
    return rms,model

In [ ]:
# load image and convert to and from NumPy array
from PIL import Image
from numpy import asarray
from skimage import transform, data, io
import cv2
# load the image

def img_to_array(path):

  img = cv2.imread(path, 1) #change 1 to 0 for gray images

  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (256,256))
  # convert image to numpy array
  #image = image.convert('RGB')
  data = asarray(img)
  #data = tf.convert_to_tensor(data)
  # summarize shape
  return data

In [ ]:
#Generar representacion bimodal en modalidad concatenacion
'''
def data_audio(train_dir, image_dir, genres):
    #Solo usaremos algunos de los generos
    #genres = os.listdir(train_dir)

    bimodal_aud = []
    
    print(genres)
    img_a=np.zeros((20, 256,256,3))
    img_b=np.zeros((20, 256,256,3))
    aud_a=np.zeros((20, 1025,1292))
    aud_b=np.zeros((20, 1025,1292))
    aud_y=np.zeros(20)
    cont_pos = 0


    for index, i in enumerate(genres):
        if(cont_pos==20):
          break 
        a_path= i 
        #todas las canciones de un mismo genero
        songs = os.listdir(train_dir+a_path)
        song_comb = list(itertools.combinations(songs, 2)) #tuplas de 2 de canciones de un mismo genero POSITIVAS
        if(len(song_comb) > 5):
          song_comb = song_comb[:5]
        for j,k in song_comb:

          spotify_id_1 = j.split('.')[0]
          spotify_id_2 = k.split('.')[0]
          image_file = spotify_id_1 + '.jpg'
          image_file_2 = spotify_id_2 + '.jpg'
          if(os.path.exists(image_dir + i + '/' + image_file) and os.path.exists(image_dir + i + '/' + image_file_2)):
            #positive samples
            #bimodal_aud.append((spotify_id_1, spotify_id_2, i))

            positive_aud_a = spectrogram(train_dir + i + '/' + j)
            positive_aud_b = spectrogram(train_dir + i + '/' + k)
            positive_img_a = img_to_array(image_dir + i + '/' + image_file)
            positive_img_b = img_to_array(image_dir + i + '/' + image_file_2)
            print(positive_aud_a.shape[1])

            if(  (positive_aud_a.shape[1] != 1292 ) or ( positive_aud_b.shape[1] != 1292 )  ):
              print('adioni')
              continue


            aud_a[cont_pos] = positive_aud_a
            aud_b[cont_pos] = positive_aud_b
            aud_y[cont_pos] = 1
            img_a[cont_pos] = positive_img_a
            img_b[cont_pos] = positive_img_b

            cont_pos = cont_pos + 1

            #negative samples

            #choose a genre different from the current one

            genres_temp = [x for x in genres if x != i]
            random_genre = random.choice(genres_temp)

            #look for a random song in the selected genre
            #folder path to random song genre
            random_songs = os.listdir(train_dir + random_genre)
            random_song = random.choice(random_songs)
            random_img = (random_song.split('.')[0]) + '.jpg'


            while(not(os.path.exists(image_dir + random_genre+'/'+random_img))):

              random_song = random.choice(random_songs)
              random_img = (random_song.split('.')[0]) + '.jpg'


            print(random_img)

            #add negative sample

            #bimodal_aud.append((spotify_id_1, random_song.split('.')[0], 0))
            aud_a[cont_pos] = spectrogram(train_dir + i + '/' + j)
            aud_b[cont_pos] = spectrogram(train_dir + random_genre + '/' + random_song)
            aud_y[cont_pos] = 0
            img_a[cont_pos] = img_to_array(image_dir + i + '/' + image_file)
            img_b[cont_pos] = img_to_array(image_dir + random_genre + '/' + random_img)


            cont_pos = cont_pos + 1

            print("--")
          else:
            print("-")
          
    return aud_a,aud_b, aud_y, img_a, img_b
    '''

In [ ]:
import glob

In [ ]:
'''

#Generar dataset representacion bimodal con autoencoders 
def data_img_spect(audio_dir, image_dir, genres):
    #Solo usaremos algunos de los generos
    #genres = os.listdir(train_dir)
    bimodal_aud = []
    img_a=np.zeros((1000, 256,256,3))
    img_b=np.zeros((1000, 256,256,3))

    y=np.zeros(1000)
    cont_pos = 0

    for index, i in enumerate(genres):

        a_path= i 
        #todas las canciones de un mismo genero
        songs = os.listdir(audio_dir+a_path)
        os.chdir(audio_dir+a_path)
        for s in glob.glob("*.jpg"):

          if(cont_pos==2000):
            break 
          spotify_id_1 = s.split('.')[0]
          #spotify_id_2 = k.split('.')[0]
          image_file = spotify_id_1 + '.jpg'
          #image_file_2 = spotify_id_2 + '.jpg'
          if(os.path.exists(image_dir + i + '/' + image_file)):


            positive_img_a = img_to_array(audio_dir + i+ '/'+s)
            positive_img_b = img_to_array(image_dir + i+ '/'+image_file)

            img_a[cont_pos] = positive_img_a
            img_b[cont_pos] = positive_img_b
            y[cont_pos] = 1
            cont_pos = cont_pos + 1
          

  
          #negative samples
          #choose a genre different from the current one

          genres_temp = [x for x in genres if x != i]
          random_genre = random.choice(genres_temp)

          #look for a random song in the selected genre
          #folder path to random song genre
          random_imgs = os.listdir(image_dir + random_genre)
          random_img = random.choice(random_imgs)
          random_img = (random_img.split('.')[0]) + '.jpg'

          #add negative sample

          img_a[cont_pos] = img_to_array(audio_dir + i+ '/'+s)

          img_b[cont_pos] = img_to_array(image_dir + random_genre + '/'+random_img)

          y[cont_pos] = 0

          cont_pos = cont_pos + 1

          
    return img_a, img_b, y
    '''

In [ ]:
#Versi[on dos corregida]
#Generar dataset representacion bimodal con autoencoders 
def data_img_spect(audio_dir, image_dir, genres):
    #Solo usaremos algunos de los generos
    #genres = os.listdir(train_dir)
    bimodal_aud = []
    img_a=np.zeros((1000, 256,256,3))
    img_b=np.zeros((1000, 256,256,3))

    y=np.zeros(1000)
    cont_pos = 0
    cont = 0

    for index, i in enumerate(genres):

        a_path= i 
        #todas las canciones de un mismo genero
        songs = os.listdir(audio_dir+a_path)
        os.chdir(audio_dir+a_path)
        print(len(songs))
        for s in songs:
          if s.endswith('.jpg'):
            cont = cont+1

            if(cont_pos==1000):
              break 
            spotify_id_1 = s.split('.')[0]
            #spotify_id_2 = k.split('.')[0]
            image_file = spotify_id_1 + '.jpg'
            #image_file_2 = spotify_id_2 + '.jpg'
            if(os.path.exists(image_dir + i + '/' + image_file)):


              positive_img_a = img_to_array(audio_dir + i+ '/'+s)
              positive_img_b = img_to_array(image_dir + i+ '/'+image_file)

              img_a[cont_pos] = positive_img_a
              img_b[cont_pos] = positive_img_b
              y[cont_pos] = 0
              cont_pos = cont_pos + 1
            

    
            #negative samples
            #choose a genre different from the current one

            genres_temp = [x for x in genres if x != i]
            random_genre = random.choice(genres_temp)

            #look for a random song in the selected genre
            #folder path to random song genre
            random_imgs = os.listdir(image_dir + random_genre)
            random_img = random.choice(random_imgs)
            random_img = (random_img.split('.')[0]) + '.jpg'

            #add negative sample

            img_a[cont_pos] = img_to_array(audio_dir + i+ '/'+s)

            img_b[cont_pos] = img_to_array(image_dir + random_genre + '/'+random_img)

            y[cont_pos] = 1

            cont_pos = cont_pos + 1

    print(cont)
    return img_a, img_b, y

In [ ]:
img_a, img_b, y = data_img_spect(spect_path, image_path, genres_y)

70
115
91
101
95
121
102
92
69
62
503


In [ ]:
#Load arrays


In [ ]:
def unison_shuffled_copies(a, b, c):
    assert len(a) == len(b)
    p = np.random.RandomState(seed=23).permutation(len(a))
    return a[p], b[p], c[p]

In [ ]:
img_a, img_b, y= unison_shuffled_copies(img_a, img_b, y)

In [ ]:
np.save("img_a.npy", img_a)
np.save("img_b.npy", img_b)
np.save("y.npy", y)


In [ ]:
path_vectors = '/content/drive/Shared drives/SpotifyMemoria/MM_dataset/vectors/'

In [ ]:
img_a = np.load(path_vectors+"img_a.npy")
img_b = np.load(path_vectors+"img_b.npy")
y = np.load(path_vectors+"y.npy")

In [ ]:
y

array([1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 1., 0., 1.

In [ ]:
  '''
  data = asarray(image)
  data = tf.convert_to_tensor(data)
  data = tf.image.resize(data, (256,256))
  '''

### Modificacion a creacion de siamese dataset



In [ ]:
#https://towardsdatascience.com/how-to-train-your-siamese-neural-network-4c6da3259463

'''
def create_batch(batch_size=256, split = "train"):
    x_anchors = np.zeros((batch_size, x_train_w_h))
    x_positives = np.zeros((batch_size, x_train_w_h))
    x_negatives = np.zeros((batch_size, x_train_w_h))
    
    if split =="train":
        data = x_train
        data_y = y_train
    else:
        data = x_test
        data_y = y_test
    
    for i in range(0, batch_size):
        # We need to find an anchor, a positive example and a negative example
        random_index = random.randint(0, data.shape[0] - 1)
        x_anchor = data[random_index]
        y = data_y[random_index]
        
        indices_for_pos = np.squeeze(np.where(data_y == y))
        indices_for_neg = np.squeeze(np.where(data_y != y))
        
        x_positive = data[indices_for_pos[random.randint(0, len(indices_for_pos) - 1)]]
        x_negative = data[indices_for_neg[random.randint(0, len(indices_for_neg) - 1)]]
        
        x_anchors[i] = x_anchor
        x_positives[i] = x_positive
        x_negatives[i] = x_negative
        
    return [x_anchors, x_positives, x_negatives]
'''

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2643133399017736196
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16154099712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8794755656647541451
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
xla_global_id: 416903419
]


In [ ]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model

from keras.callbacks import EarlyStopping
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten,Concatenate,Reshape
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop

import tensorflow as tf

In [ ]:
train_dir = "train"
test_dir = "test"
batch_size = 5
epochs = 30
input_img_dim = (256,256,3)
input_aud_dim = (1025, 1292,1)

#img_a,aud_a,img_b,aud_b,labels = data_generate(train_dir)
labels = aud_y


In [ ]:
'''
opt,model = siamese_network(input_img_dim,input_aud_dim)

model.compile(loss=contrastive_loss, optimizer=opt)
model.summary()

es = EarlyStopping(monitor='val_acc', mode='min', verbose=1)
model.fit([tf.convert_to_tensor(img_a, dtype=tf.float32),tf.convert_to_tensor(aud_a, dtype=tf.float32),tf.convert_to_tensor(img_b, dtype=tf.float32),tf.convert_to_tensor(aud_b, dtype=tf.float32)], tf.convert_to_tensor(labels, dtype=tf.float32), validation_split=.25,batch_size=batch_size, verbose=True, epochs=epochs, callbacks=[es])
#model.fit([img_a,aud_a,img_b,aud_b], labels, validation_split=.25,batch_size=batch_size, verbose=True, epochs=epochs, callbacks=[es])
'''

### Siamese bimodal with autoencoders extraction

In [ ]:
#Autoencoders para imágenes
'''

def conv_block(input, num_filters):
  
  x = Conv2D(num_filters, 3, padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  x = Conv2D(num_filters, 3, padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  return x

def encoder_block(input, num_filters):
  x = conv_block(input, num_filters)
  p = MaxPooling2D(pool_size=(2, 2))(x)
  return x, p


def decoder_block(input, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(input)
  x = conv_block(x, num_filters)
  return x


def build_encoder(input_image):
  s1, p1 = encoder_block(input_image, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  encoded = conv_block(p4, 1024)
  encoder_output = layers.Dense(100, activation='linear')(encoded)
  return encoder_output

def build_decoder(encoded):
  d1 = decoder_block(encoded, 256)
  d2 = decoder_block(d1, 128)
  d3 = decoder_block(d2, 64)

  decoded = Conv2D(3, 3, padding='same', activation='sigmoid')(d3)
  return decoded

def build_auto_encoder(input_shape):
  input_image = Input(shape=input_shape)
  autoencoder = Model(input_image, build_decoder(build_encoder(input_image)))

  return(autoencoder)
'''

#def siamese_network ():




"\ndef build_decoder(encoded):\n  d1 = decoder_block(encoded, 256)\n  d2 = decoder_block(d1, 128)\n  d3 = decoder_block(d2, 64)\n\n  decoded = Conv2D(3, 3, padding='same', activation='sigmoid')(d3)\n  return decoded\n\ndef build_auto_encoder(input_shape):\n  input_image = Input(shape=input_shape)\n  autoencoder = Model(input_image, build_decoder(build_encoder(input_image)))\n\n  return(autoencoder)\n"

In [ ]:
'''
# example of loading the resnet50 model
from tensorflow.keras.applications.resnet50 import ResNet50
# load model
# summarize the model

def build_encoder(input_image, name):
  encoder = ResNet50(include_top=False, pooling='avg', weights="imagenet")(input_image)
  encoder.name = name
  return encoder


'''

In the original paper that proposed dropout layers, by Hinton (2012), dropout (with p=0.5) was used on each of the fully connected (dense) layers before the output; it was not used on the convolutional layers. This became the most commonly used configuration.

More recent research has shown some value in applying dropout also to convolutional layers, although at much lower levels: p=0.1 or 0.2. Dropout was used after the activation function of each convolutional layer: CONV->RELU->DROP.

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:
def resnet_rename():
  encoder_1 = InceptionResNetV2(include_top=False, pooling='avg')
  encoder_1._name = 'InceptionResNetV2_2'
  return encoder_1


In [ ]:
def siamese(input_dim_img):
    
    img_a = Input(shape=input_dim_img)  #imagen de cover art
    img_b = Input(shape=input_dim_img) #imagen de espectrograma

    #encoder_1 = build_encoder(img_a, "resnet1")
    #encoder_2 = build_encoder(img_b, "resnet2")

    encoder_1 = InceptionResNetV2(include_top=False, pooling='avg')(img_a)
    encoder_2 = resnet_rename()(img_b)

    #conc = Concatenate(axis=-1)([encoder_1, encoder_2])
    #conc = Concatenate(axis=-1)([input_1, input_2, input_3])

    #print(conc.shape)

    # Capa densa
    distance = distance = Lambda(euclidean_distance, output_shape = eucl_dist_output_shape)([encoder_1, encoder_2])

    ##flatten = Flatten()(distance)
    out = Dense(1, activation='sigmoid')(distance)
    print(distance)


    #classifier.add(Dense(units = 32, activation = 'relu'))
    # Modelo

    ##model = Model([img_a, img_b], out)
    rms = RMSprop()
    model = Model([img_a, img_b], out)

    #model.compile(loss="binary_crossentropy", metrics=['acc',auroc], optimizer=Adam(l_rate)) #0.00001

    #model.compile(loss=contrastive_loss, optimizer='Adam') #0.00001

    #distance = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([feat_vecs_a ,feat_vecs_b])
    
    return rms,model

In [ ]:
import cv2 #probar que autoencoder este funcionando
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
img_name = '7GsoKl0RTc7jFU3P2Ya3kK.jpg'
img_path = '/content/drive/Shared drives/SpotifyMemoria/MM_dataset/image/ambient/'
SIZE = 256

img_data = []

In [ ]:
img_path+img_name

'/content/drive/Shared drives/SpotifyMemoria/MM_dataset/image/ambient/7GsoKl0RTc7jFU3P2Ya3kK.jpg'

In [ ]:
img = cv2.imread(img_path+img_name, 1) #change 1 to 0 for gray images

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (SIZE,SIZE))
img_data.append(img_to_array(img))

img_array = np.reshape(img_data, (len(img_data), SIZE, SIZE, 3))
img_array = img_array.astype('float32') / 255.
#img_t = tf.convert_to_tensor(img_array)



In [ ]:
#train autoencoder
opt, model = siamese(img.shape)
model.compile(loss=contrastive_loss, optimizer=opt, metrics=['accuracy'])
print(model.summary())


219070464/219055592 [==============================] - 2s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='lambda/Sqrt:0', description="created by layer 'lambda'")
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_6 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                

In [ ]:
#K.set_value(model.optimizer.learning_rate, 0.001)
model.fit([img_a,img_b],y, validation_split=.25,batch_size=30, verbose=True, epochs=100)#, callbacks=[es])

Epoch 1/100
25/25 [==============================] - 97s 1s/step - loss: 0.3055 - accuracy: 0.4920 - val_loss: 0.4760 - val_accuracy: 0.5240
Epoch 2/100
25/25 [==============================] - 23s 922ms/step - loss: 0.2587 - accuracy: 0.4920 - val_loss: 0.4676 - val_accuracy: 0.5240
Epoch 3/100
25/25 [==============================] - 23s 944ms/step - loss: 0.2612 - accuracy: 0.4947 - val_loss: 0.4709 - val_accuracy: 0.5240
Epoch 4/100
25/25 [==============================] - 23s 940ms/step - loss: 0.2552 - accuracy: 0.4947 - val_loss: 0.3825 - val_accuracy: 0.5240
Epoch 5/100
25/25 [==============================] - 23s 936ms/step - loss: 0.2551 - accuracy: 0.4933 - val_loss: 0.3443 - val_accuracy: 0.5840
Epoch 6/100
25/25 [==============================] - 23s 934ms/step - loss: 0.2519 - accuracy: 0.4973 - val_loss: 0.3733 - val_accuracy: 0.5080
Epoch 7/100
25/25 [==============================] - 23s 927ms/step - loss: 0.2530 - accuracy: 0.5080 - val_loss: 0.3345 - val_accuracy: 0.

In [ ]:
#model.fit([img_a,img_b],y, validation_split=.25,batch_size=25, verbose=True, epochs=100)#, callbacks=[es])
#tf.convert_to_tensor(labels, dtype=tf.float32)

In [ ]:
results_train = model.predict([img_a,img_b])

In [ ]:
results_train.shape

In [ ]:
results_train

In [ ]:
y

In [ ]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state(y, results_train)
print("Binary Accuracy: ", m.result().numpy())

Binary Accuracy:  0.791


In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 1.1 MB/s 


In [ ]:
#!pip install faiss-gpu
import faiss                            
faiss_index = faiss.IndexFlatL2(1000)   # build the index

# storing the image representations
im_indices = []
with torch.no_grad():
    for f in glob.glob(os.path.join(PATH_TRAIN, '*/*')):
        im = Image.open(f)
        im = im.resize((224,224))
        im = torch.tensor([val_transforms(im).numpy()]).cuda()
    
        preds = model(im)
        preds = np.array([preds[0].cpu().numpy()])
        faiss_index.add(preds) #add the representation to index
        im_indices.append(f)   #store the image name to find it later on

# Retrieval with a query image
with torch.no_grad():
    for f in os.listdir(PATH_TEST):
        # query/test image
        im = Image.open(os.path.join(PATH_TEST,f))
        im = im.resize((224,224))
        im = torch.tensor([val_transforms(im).numpy()]).cuda()
    
        test_embed = model(im).cpu().numpy()
        _, I = faiss_index.search(test_embed, 5)
        print("Retrieved Image: {}".format(im_indices[I[0][0]]))